In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk # natural language tool kit

In [29]:
initialFrame = pd.read_csv('train.tsv', delimiter = '\t');

In [30]:
initialFrame

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [31]:
# Cleans the initial frame
def cleanInitialFrame(df):
    cleanList = [] # list to grow
    currentSentence = 0 # tracks current sentence
    sentenceIDs = {0} 
    # Iterate row by row
    for index, row in df.iterrows():
        # If it's the first element, add to list
        if (row['SentenceId'] == currentSentence):
            continue
        else:
            cleanList.append([row['PhraseId'], row['SentenceId'], row['Phrase'], row['Sentiment']]);
            currentSentence = row['SentenceId']
    
    # Return a clean frame
    return pd.DataFrame(cleanList, columns = ['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'])

In [32]:
df = cleanInitialFrame(initialFrame)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,64,2,"This quiet , introspective and entertaining in...",4
2,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
3,117,4,A positively thrilling combination of ethnogra...,3
4,157,5,Aggressive self-glorification and a manipulati...,1


In [33]:
def lowerAllPhrases(df):
    phrases_list = list(df['Phrase'])

    for i in range(len(phrases_list)):
        phrases_list[i] = phrases_list[i].lower()
    count = 0;
    for index, row in df.iterrows():
        df.at[index,'Phrase'] = phrases_list[count]
        count += 1

    return df

In [34]:
lowerAllPhrases(df)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1


In [35]:
# Remove non-ascii characters using str.replace()
def asciiClean(df):
    # iterate row by row
    for index, row in df.iterrows():
        old_str = row['Phrase']
        new_str = (old_str.encode('ascii','ignore')).decode()
        df.at[index, 'Phrase'] = new_str

In [36]:
asciiClean(df)
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1


In [37]:
df[0:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant 's work , i suspe...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1
5,167,6,a comedy-drama of nearly epic proportions root...,4
6,199,7,"narratively , trouble every day is a plodding ...",1
7,214,8,"the importance of being earnest , so thick wit...",3
8,248,9,but it does n't leave you with much .,1
9,260,10,you could hate it for the same reason .,1


In [38]:
import re
def removeSpaces(df):
    for index, row in df.iterrows():
        # df['Phrase'] = df['Phrase'].replace([row['Phrase']], re.sub(r'\s+\'', "'", row['Phrase']))
        df.at[index,'Phrase'] = re.sub(r'\s+\'', "'", row['Phrase'])

In [39]:
removeSpaces(df)

In [40]:
df[0:10]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,a series of escapades demonstrating the adage ...,1
1,64,2,"this quiet , introspective and entertaining in...",4
2,82,3,"even fans of ismail merchant's work , i suspec...",1
3,117,4,a positively thrilling combination of ethnogra...,3
4,157,5,aggressive self-glorification and a manipulati...,1
5,167,6,a comedy-drama of nearly epic proportions root...,4
6,199,7,"narratively , trouble every day is a plodding ...",1
7,214,8,"the importance of being earnest , so thick wit...",3
8,248,9,but it does n't leave you with much .,1
9,260,10,you could hate it for the same reason .,1


In [ ]:
import contractions
contractions.add('n\'t', 'not')
def expandContractions(df):
    for index, row in df.iterrows():
        phrase = []
        for i in row['Phrase'].split():
            phrase.append(contractions.fix(i))
        string_version = ' '.join(phrase)
        df.at[index, 'Phrase'] = string_version            

In [ ]:
expandContractions(df)

In [41]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muchlogic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/muchlogic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
def removeStopWords(df):
    
    phrases_list = list(df['Phrase'])
    stop_words = set(stopwords.words('english'))

    for i in range(len(phrases_list)):
        word_tokens = word_tokenize(phrases_list[i])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        phrases_list[i] = filtered_sentence
    
    for i in range(len(phrases_list)):
        phrases_list[i] = TreebankWordDetokenizer().detokenize(phrases_list[i])
    
    count = 0
    for index, row in df.iterrows():
        df['Phrase'] = df['Phrase'].replace([row['Phrase']], phrases_list[count])
        df.at[index, 'Phrase'] = phrases_list[count]
        count += 1

    return df      

In [43]:
df = removeStopWords(df)
df

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,64,2,"quiet, introspective entertaining independent ...",4
2,82,3,"even fans ismail merchant's work, suspect, wou...",1
3,117,4,positively thrilling combination ethnography i...,3
4,157,5,aggressive self-glorification manipulative whi...,1
...,...,...,...,...
8524,155985,8540,... either're willing go claustrophobic concep...,2
8525,155998,8541,"despite annoyances, capable clayburgh tambor r...",2
8526,156022,8542,-lrb- tries -rrb- parody genre's already joke ...,1
8527,156032,8543,movie's downfall substitute plot personality.,1


In [1]:
# TO DO:
# * preprocessing
#     - replace all grammar with spaces
#     - lemmatize words
# * reduce to multi-dimensional vector
#     - We have options including: Bag of words
#     - BERT
#     - TF-IDF
# * classification
#     - try a bunch of classifiers
# * graphing
#     - graph the outputs of our classifiers
# * presentation
#     - create a presentation